# <center>Project for Foundations of Computer Science</center>
### <center>University of Milano-Bicocca</center>
<center>Matteo Corona - Costanza Pagnin</center>

### 0. Preliminary steps
### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import ast
import re

### Reading *.csv* files from GitHub Repository

In [2]:
travel=pd.read_csv('https://raw.githubusercontent.com/CoroTheBoss/CS-project/main/dogTravel.csv', index_col=0)
dog=pd.read_csv('https://raw.githubusercontent.com/CoroTheBoss/CS-project/main/dogs.csv')
nst=pd.read_csv('https://raw.githubusercontent.com/CoroTheBoss/CS-project/main/NST-EST2021-POP.csv', header=None)

### 1. Extract all dogs with status that is *not adoptable*

Some values were off by one column so they had to be properly shifted

In [3]:
# Shifting values
dog.loc[dog['status']!='adoptable','status':'accessed'] = dog.loc[dog['status']!='adoptable','status':'accessed'].shift(periods=1, axis="columns")

In [4]:
# Cheching all possible values in status
dog["status"].unique()

array(['adoptable', nan], dtype=object)

Since there are two different values, the NaN values refers to the *not adoptable* dogs

In [5]:
# Replacing NaN values
dog.loc[dog.status != 'adoptable', ['status']] = 'not adoptable'
# Printing the first not adoptable dogs to visualize the data
dog.loc[dog.status != 'adoptable', ['id', 'status']].head()

,id,status
644,41330726,not adoptable
5549,38169117,not adoptable
10888,45833989,not adoptable
11983,45515547,not adoptable
12495,45294115,not adoptable


In [6]:
print("There are", len(dog[dog.status != 'adoptable']) ,"dogs with status that is not adoptable" )

There are 33 dogs with status that is not adoptable


### 2. For each (primary) breed, determine the number of dogs

In [7]:
# Checking if all dogs have a primary key
dog[dog.breed_primary.isna()]

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost


In [8]:
# Checking if all dogs have an id
dog[dog.id.isna()]

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost


In [9]:
# Grouping dogs by their primary key and counting them
dog.groupby('breed_primary')['id'].count()

breed_primary
Affenpinscher                         17
Afghan Hound                           4
Airedale Terrier                      19
Akbash                                 3
Akita                                181
                                    ... 
Wirehaired Pointing Griffon            1
Wirehaired Terrier                    60
Xoloitzcuintli / Mexican Hairless     11
Yellow Labrador Retriever            158
Yorkshire Terrier                    360
Name: id, Length: 216, dtype: int64

### 3. For each (primary) breed, determine the ratio between the number of dogs of `Mixed Breed` and those not of Mixed Breed. Hint: look at the `secondary_breed`.

In [10]:
breed_tab = dog.groupby(['breed_primary','breed_mixed'])['id'].count()
breed_tab = breed_tab.unstack()
breed_tab.columns = ['not_mixed', 'mixed']
breed_tab[np.isnan(breed_tab)] = 0
breed_tab

,not_mixed,mixed
breed_primary,,
Affenpinscher,12.0,5.0
Afghan Hound,0.0,4.0
Airedale Terrier,2.0,17.0
Akbash,1.0,2.0
Akita,98.0,83.0
...,...,...
Wirehaired Pointing Griffon,0.0,1.0
Wirehaired Terrier,15.0,45.0
Xoloitzcuintli / Mexican Hairless,6.0,5.0


In [21]:
breed_tab["mixed_%"] = round(100 * breed_tab["mixed"] / (breed_tab["mixed"] + breed_tab["not_mixed"]), 1)
breed_tab["not_mixed_%"] = round(100 * breed_tab["not_mixed"] / (breed_tab["mixed"] + breed_tab["not_mixed"]), 1)
breed_tab["ratio"] = breed_tab["mixed"] / breed_tab["not_mixed"]
breed_tab

,not_mixed,mixed,mixed_%,not_mixed_%,ratio
breed_primary,,,,,
Affenpinscher,12.0,5.0,29.4,70.6,0.416667
Afghan Hound,0.0,4.0,100.0,0.0,inf
Airedale Terrier,2.0,17.0,89.5,10.5,8.500000
Akbash,1.0,2.0,66.7,33.3,2.000000
Akita,98.0,83.0,45.9,54.1,0.846939
...,...,...,...,...,...
Wirehaired Pointing Griffon,0.0,1.0,100.0,0.0,inf
Wirehaired Terrier,15.0,45.0,75.0,25.0,3.000000
Xoloitzcuintli / Mexican Hairless,6.0,5.0,45.5,54.5,0.833333


### 4. For each (primary) breed, determine the earliest and the latest `posted` timestamp.



In [12]:
dog.posted = pd.to_datetime(dog.posted)
time_tab = dog.groupby('breed_primary')[['posted']].min()
time_tab['postedmin'] = dog.groupby('breed_primary')[['posted']].max()
time_tab.columns = ['erliest_posted_timestamp', 'latest_posted_timestamp']
time_tab

,erliest_posted_timestamp,latest_posted_timestamp
breed_primary,,
Affenpinscher,2012-03-08 10:27:33+00:00,2019-09-14 10:10:51+00:00
Afghan Hound,2017-06-29 23:28:51+00:00,2019-07-27 00:38:48+00:00
Airedale Terrier,2014-06-13 12:59:36+00:00,2019-09-19 18:40:39+00:00
Akbash,2019-07-21 00:35:59+00:00,2019-08-23 17:11:04+00:00
Akita,2012-03-03 09:31:08+00:00,2019-09-20 15:19:57+00:00
...,...,...
Wirehaired Pointing Griffon,2016-06-29 20:03:55+00:00,2016-06-29 20:03:55+00:00
Wirehaired Terrier,2012-11-27 14:07:54+00:00,2019-09-19 22:52:45+00:00
Xoloitzcuintli / Mexican Hairless,2007-02-01 00:00:00+00:00,2019-09-08 11:15:54+00:00


### 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

In [13]:
state_tab = dog.groupby(['contact_state','sex'])['id'].count()
state_tab = state_tab.unstack()
state_tab[np.isnan(state_tab)] = 0
state_tab["sex_imbalance"] = state_tab["Male"] - state_tab["Female"]
state_tab.head()

sex,Female,Male,Unknown,sex_imbalance
contact_state,,,,
AK,7.0,8.0,0.0,1.0
AL,716.0,712.0,0.0,-4.0
AR,351.0,344.0,0.0,-7.0
AZ,1067.0,1181.0,1.0,114.0
CA,777.0,887.0,0.0,110.0


In [14]:
print("The state with the highest sex imbalance is Ohio.")
state_tab.loc[state_tab["sex_imbalance"] == state_tab["sex_imbalance"].max()]

The state with the highest sex imbalance is Ohio.


sex,Female,Male,Unknown,sex_imbalance
contact_state,,,,
OH,1234.0,1439.0,0.0,205.0


### 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

In [15]:
round(dog.groupby(['age','size'])[['stay_duration','stay_cost']].mean(), 2)

stay_duration  stay_cost
age    size                                 
Adult  Extra Large          89.02     232.59
       Large                89.53     238.66
       Medium               89.42     238.26
       Small                89.41     238.97
Baby   Extra Large          87.03     237.18
       Large                89.70     238.70
       Medium               89.58     237.11
       Small                89.96     239.08
Senior Extra Large          88.86     235.23
       Large                88.98     237.51
       Medium               89.81     238.51
       Small                89.07     238.28
Young  Extra Large          90.59     245.84
       Large                90.10     238.15
       Medium               89.52     239.30
       Small                89.81     241.54

### 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [16]:
travel_tab = travel.groupby(['id'], as_index=False)[['contact_city']].count()
travel_tab.columns = ["id", "count"]
travel_tab_3 = travel_tab[travel_tab["count"] > 2]
pd.merge(travel_tab_3,dog[["id", "breed_primary", "breed_secondary", "breed_mixed", "breed_unknown"]],
         how='left', on=["id"])

,id,count,breed_primary,breed_secondary,breed_mixed,breed_unknown
0,16657005,4,Pit Bull Terrier,NaN,True,False
1,20905974,5,Chow Chow,English Springer Spaniel,True,False
2,24894870,4,Hound,Terrier,True,False
3,24894894,4,Hound,Terrier,True,False
4,33218331,7,Alaskan Malamute,Siberian Husky,True,False
...,...,...,...,...,...,...
558,46042569,3,Labrador Retriever,NaN,False,False
559,46042587,3,Labrador Retriever,NaN,False,False
560,46042618,3,Labrador Retriever,NaN,False,False
561,46043099,3,Labrador Retriever,NaN,False,False


In [17]:
print("There are", len(travel_tab_3) ,"dogs involved in at least three travels" )

There are 563 dogs involved in at least three travels


### 8. Fix the `travels` table so that the correct state is computed from  the `manual` and the `found` fields. If `manual` is not missing, then it overrides what is stored in `found`.

In [17]:
i = 0
for elem in travel["manual"]:
    if pd.isnull(elem) == False:
        travel["found"].values[i] = elem
    i = i + 1
travel

,id,contact_city,contact_state,description,found,manual,remove,still_there
index,,,,,,,,
0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Bahamas,Bahamas,NaN,NaN
2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Maryland,Maryland,NaN,NaN
3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
6189,40492179,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,NaN,True,NaN
6190,45799729,Eagle Mountain,UT,Shiny is an approximately 4-6-year-old spayed ...,Wyoming,NaN,NaN,NaN
6191,34276515,Newnan,GA,Yanni is a Male Great Pyrenees that we rescue...,Yazmin,NaN,True,NaN


### 9. For each state, compute the ratio between the number of travels and the population.

In [18]:
# Fixing a value in travels: 17325 refers to Pennsylvania (PA)
travel["contact_state"] = travel["contact_state"].replace('17325','PA')
# Opening and reading the file (found on the internet) which contains the state abbreviations
file = open("abbreviations.txt", "r")
contents = file.read()
# Converting the file into a dictionary
abb = ast.literal_eval(contents)
file.close()
# Naming nst columns (the file was without header)
nst.columns = ["state", "population"]
# Substituting state names with abbreviations
nst = nst.replace({"state": abb})
# Converting population column into numeric values
nst["population"] = nst["population"].str.replace('.', '', regex=True)
nst.head()

,state,population
0,AL,5024279
1,AK,733391
2,AZ,7151502
3,AR,3011524
4,CA,39538223


In [35]:
state_tab_2 = travel.groupby(['contact_state'], as_index=False)['id'].count()
state_tab_2.columns = ["state", "travels"]
state_tab_2 = pd.merge(nst,state_tab_2[["state","travels"]],
         how='left', on=["state"])
state_tab_2 = state_tab_2.fillna(0)
state_tab_2["population"] = state_tab_2["population"].astype(float)
state_tab_2["ratio"] = state_tab_2["travels"] / state_tab_2["population"]
state_tab_2.head()

,state,population,travels,ratio
0,AL,5024279.0,75.0,1.492751e-05
1,AK,733391.0,0.0,0.000000e+00
2,AZ,7151502.0,70.0,9.788154e-06
3,AR,3011524.0,10.0,3.320578e-06
4,CA,39538223.0,28.0,7.081755e-07


### 10. For each dog, compute the number of days from the `posted` day to the day of last access.

### 11. Partition the dogs according to the number of weeks from the `posted` day to the day of last access.

### 12. Find for duplicates in the `dogs` dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.